## Loading neccessary libraries

In [1]:
import torch
from torch.utils.data import Dataset
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
# from transformers import AutoTokenizer, BertTokenizer, DistilBertModel
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification


## Loading the data

In [2]:
url_data = 'https://raw.githubusercontent.com/TeodorRusKvi/Tekstanalyse/main/git_NLP_data/'

df = pd.read_csv(url_data + 'new_df.csv')

In [3]:
y_liberal = pd.read_csv(url_data + 'y_liberal.csv')
y_data = pd.read_csv(url_data + 'y_data.csv')
X_text = pd.read_csv(url_data + 'X_text.csv')

In [4]:
X_text= X_text['Processed'].tolist()
y = y_liberal

In [5]:
text_train, text_test, y_train, y_test = train_test_split(
    X_text, 
    y, 
    test_size=0.2, 
    stratify=y,
    random_state=42)

## 4. Processing the data

In [6]:
from torch.utils.data import Dataset
import torch

class TextDataset(Dataset):
    def __init__(self, text, labels, tokenizer, max_length):
        self.tokenized_txt = tokenizer(
            text,
            max_length=max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        self.labels = torch.tensor(labels)
      
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {
            key: self.tokenized_txt[key][idx] for key in self.tokenized_txt
        }
        item['labels'] = self.labels[idx]
        return item

## 5. Initialize the model

In [7]:
Distilbert = 'distilbert-base-uncased-finetuned-sst-2-english'
Roberta = 'roberta-base'
Bert = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(Distilbert)
model = AutoModelForSequenceClassification.from_pretrained(Distilbert, num_labels=2)

In [8]:
dataset_train = TextDataset(
    text_train, 
    y_train.values, 
    tokenizer, 
    max_length=20
)

dataset_test = TextDataset(
    text_test, 
    y_test.values, 
    tokenizer, 
    max_length=20
)

from torch.utils.data import DataLoader

train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False)

In [9]:
dataset_test[9]

{'input_ids': tensor([  101,  5976,  7394, 21887, 23350, 17404, 24006,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor([1])}

## 6. Training Setup

In [10]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1_result = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    balanced_acc = balanced_accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'balanced_accuracy': balanced_acc,
        'f1_score': f1_result
        }

In [11]:
# create a dir to store model checkpoints
my_dir = 'roberta_checkpoints'

### Using the Hugging Face Trainer

In [12]:
from torch.optim import AdamW
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader



# Optimizer setup
optimizer = AdamW(model.parameters(), lr=0.001)

# Scheduler setup
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0)

# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# define training arguments
training_args = TrainingArguments(
    output_dir=my_dir,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    learning_rate=0.0001,
    lr_scheduler_type='cosine'
)

In [13]:
train_dataloader = DataLoader(dataset_train, batch_size=64, shuffle=True, collate_fn=data_collator)

for batch in train_dataloader:
    print(f"Input IDs shape: {batch['input_ids'].shape}")
    print(f"Labels shape: {batch['labels'].shape}")
    break

Input IDs shape: torch.Size([64, 20])
Labels shape: torch.Size([64, 1])


In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    #optimizer=(optimizer, scheduler)
)

c:\Users\bugat\anaconda3\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
# train the model
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: teodor-ruskvi. Use `wandb login --relogin` to force relogin


  0%|          | 0/805 [00:00<?, ?it/s]

  0%|          | 0/41 [00:00<?, ?it/s]

{'eval_loss': 0.493280827999115, 'eval_accuracy': 0.7681835861532478, 'eval_balanced_accuracy': 0.7140781369264693, 'eval_f1_score': 0.757417492845584, 'eval_runtime': 67.9938, 'eval_samples_per_second': 37.812, 'eval_steps_per_second': 0.603, 'epoch': 1.0}


  0%|          | 0/41 [00:00<?, ?it/s]

...

  0%|          | 0/41 [00:00<?, ?it/s]

{'eval_loss': 0.9103769063949585, 'eval_accuracy': 0.7829638273045507, 'eval_balanced_accuracy': 0.7485688236790773, 'eval_f1_score': 0.7794306255116292, 'eval_runtime': 68.2721, 'eval_samples_per_second': 37.658, 'eval_steps_per_second': 0.601, 'epoch': 4.0}


  0%|          | 0/41 [00:00<?, ?it/s]

{'eval_loss': 0.9591248035430908, 'eval_accuracy': 0.7833527810190587, 'eval_balanced_accuracy': 0.7501232401831905, 'eval_f1_score': 0.7801565590514468, 'eval_runtime': 65.1313, 'eval_samples_per_second': 39.474, 'eval_steps_per_second': 0.629, 'epoch': 5.0}
{'train_runtime': 5129.6124, 'train_samples_per_second': 10.023, 'train_steps_per_second': 0.157, 'train_loss': 0.25055436081027394, 'epoch': 5.0}


TrainOutput(global_step=805, training_loss=0.25055436081027394, metrics={'train_runtime': 5129.6124, 'train_samples_per_second': 10.023, 'train_steps_per_second': 0.157, 'train_loss': 0.25055436081027394, 'epoch': 5.0})

In [19]:
# Save the model to a specified directory
Dir = r"\models\Bert\"
model_directory = Dir + "model"
trainer.save_model(model_directory)

# If you also want to save the tokenizer used during training:
tokenizer_directory = Dir + 'tokenizer_dir'
trainer.tokenizer.save_pretrained(tokenizer_directory)

SyntaxError: unterminated string literal (detected at line 2) (1507962286.py, line 2)